# Garage Classifcation Model

## Goal: Given coordinate points along a path, classify whether a person is headed towards the garage.

---

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../simulations/csv_paths/cottage_grove_paths.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 62 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Unnamed: 0                      92 non-null     object 
 1   latitude_1                      92 non-null     float64
 2   longitude_1                     92 non-null     float64
 3   altitude_relative_to_ground_1   92 non-null     float64
 4   latitude_2                      92 non-null     float64
 5   longitude_2                     92 non-null     float64
 6   altitude_relative_to_ground_2   92 non-null     float64
 7   latitude_3                      92 non-null     float64
 8   longitude_3                     92 non-null     float64
 9   altitude_relative_to_ground_3   92 non-null     float64
 10  latitude_4                      92 non-null     float64
 11  longitude_4                     92 non-null     float64
 12  altitude_relative_to_ground_4   92 non

In [5]:
df.head()

,Unnamed: 0,latitude_1,longitude_1,altitude_relative_to_ground_1,latitude_2,longitude_2,altitude_relative_to_ground_2,latitude_3,longitude_3,altitude_relative_to_ground_3,...,latitude_18,longitude_18,altitude_relative_to_ground_18,latitude_19,longitude_19,altitude_relative_to_ground_19,latitude_20,longitude_20,altitude_relative_to_ground_20,enter-parking
0,10_A.xml,36.111200,-115.141654,0.0,36.111206,-115.141554,0.0,36.111211,-115.141400,0.0,...,36.110529,-115.140651,0.0,36.110555,-115.140550,0.0,36.110629,-115.140521,0.0,1
1,10_A_Missed.xml,36.109699,-115.140636,0.0,36.109756,-115.140660,0.0,36.109795,-115.140685,0.0,...,36.110881,-115.140836,0.0,36.110995,-115.140851,0.0,36.111160,-115.140860,0.0,0
2,10_B.xml,36.110692,-115.141014,0.0,36.110658,-115.140986,0.0,36.110627,-115.140952,0.0,...,36.110594,-115.139715,0.0,36.110633,-115.139636,0.0,36.110667,-115.139667,0.0,1
3,10_B_Missed.xml,36.110448,-115.139256,0.0,36.110419,-115.139264,0.0,36.110380,-115.139272,0.0,...,36.109740,-115.139144,0.0,36.109708,-115.139112,0.0,36.109650,-115.139100,0.0,0
4,11_A.xml,36.109695,-115.140862,0.0,36.109748,-115.140871,0.0,36.109808,-115.140878,0.0,...,36.110561,-115.140546,0.0,36.110599,-115.140523,0.0,36.110638,-115.140528,0.0,1


In [6]:
# Get First Coordinate Pair Only -> Columns [1, 6]
X = df[df.columns[1:7]]
label = df['enter-parking'].values

In [7]:
print(X)
print(label)

    latitude_1  longitude_1  altitude_relative_to_ground_1  latitude_2  \
0    36.111200  -115.141654                            0.0   36.111206   
1    36.109699  -115.140636                            0.0   36.109756   
2    36.110692  -115.141014                            0.0   36.110658   
3    36.110448  -115.139256                            0.0   36.110419   
4    36.109695  -115.140862                            0.0   36.109748   
..         ...          ...                            ...         ...   
87   36.110949  -115.138459                            0.0   36.110880   
88   36.109628  -115.141544                            0.0   36.109697   
89   36.109490  -115.140931                            0.0   36.109541   
90   36.109708  -115.140845                            0.0   36.109824   
91   36.110942  -115.138463                            0.0   36.110884   

    longitude_2  altitude_relative_to_ground_2  
0   -115.141554                            0.0  
1   -115.1406

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,label,test_size=0.25,random_state=101)

In [ ]:
# Scale data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(X_train)

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
